In [13]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Configurações
image_size = 128
num_classes = 10  # Fashion MNIST tem 10 classes

# Transformações para as imagens
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.Grayscale(num_output_channels=3),  # Converte para 3 canais
    transforms.ToTensor(),
])

# Carregar o dataset Fashion MNIST
dataset = datasets.FashionMNIST(root='data', train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Carregar o modelo ResNet18 pré-treinado e ajustar para o número de classes
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.eval()  # Definir o modelo para o modo de avaliação

# Função para extrair características de uma imagem
def extract_features(image_path, model):
    image = Image.open(image_path).convert('RGB')  # Assegurar que a imagem é RGB
    image = transform(image).unsqueeze(0)  # Adicionar batch dimension
    with torch.no_grad():
        features = model(image)
    return features.numpy()

# Função para calcular similaridade entre imagens
def find_similar_images(query_features, all_features):
    similarities = cosine_similarity(query_features, all_features)
    return similarities

In [14]:
def get_all_features(dataloader, model):
    all_features = []
    with torch.no_grad():
        for images, _ in dataloader:
            features = model(images)
            all_features.append(features.numpy())
    return np.concatenate(all_features, axis=0)

# Extraindo características de todas as imagens no dataset
all_image_features = get_all_features(dataloader, model)

In [16]:
def recommend_images(query_image_path):
    query_features = extract_features(query_image_path, model)
    similarities = find_similar_images(query_features, all_image_features)
    recommended_indices = similarities.argsort()[::-1]
    
    # Exibir as imagens recomendadas
    plt.figure(figsize=(10, 10))
    for i, idx in enumerate(recommended_indices[:5]):  # Mostrar as 5 imagens mais similares
        plt.subplot(1, 5, i + 1)
        plt.imshow(dataset[idx][0].permute(1, 2, 0))  # Reordenar as dimensões da imagem
        plt.axis('off')
    plt.show()

# Substitua 'path_to_your_image.jpg' pelo caminho da imagem que deseja usar
recommend_images(r"C:\Users\Bruno Birello\Downloads\Oculos.jpg")

ValueError: only one element tensors can be converted to Python scalars